In [1]:
import tensorflow as tf
from gensim.models import Word2Vec
import pandas as pd
from textblob import TextBlob
import numpy as np

c:\users\princ3\anaconda3\envs\tfconda\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### Importing the word2vec models

In [2]:
word_vec = Word2Vec.load('model.bin')    # word embedding
# char_vec = Word2Vec.load('model_c.bin')  # charector embedding

### Importing the dataset

In [3]:
df_contxt = pd.read_pickle('./input/squad_contxt.pkl') # context
df_qas = pd.read_pickle('./input/squad_qas.pkl')       # question and answers

### Preprocessing data

In [4]:
class prep_data():
    """
    
    """
    def __init__(self,contxt = df_contxt,qas = df_qas, word_e = word_vec):
        
        self.contxt = contxt
        self.qas = qas
        self.char_e = char_e
        self.word_e = word_e

    def C2V(self,passage):   # this function converts context 2 vector with <EOS> tag
        tb_p = TextBlob(passage)
        p2v = []
        for i in tb_p.sentences:
            words = TextBlob(str(i)+" E-O-S").words
            for j in words:
                p2v.append(word_vec[j])
        return np.array(p2v)
    
    def QA2V(self,question_no):  # this function converts question 2 vector
        tb_a = TextBlob(self)
        
        

###  Building the model

In [5]:
class DMN_QA():
    """
    the word vectors have size 100
    the max no of words in a sentence is 120
    the answers are fed as starting and ending of the span
    """
    
    def __init__(self):
        self.MAX_WORD_NO = 120
        self.WORD_VEC_LEN = 100
    
    def build(self):
        # variables and placeholders
        Passage = tf.placeholder(tf.float32)
        EOS_Tag = tf.placeholder(tf.float32)
        Question = tf.placeholder(tf.float32)
        Answer = tf.placeholder(tf.float32)     
                 
        # weigts for scalar kernel and memory vector
        W_b = tf.Variable(tf.float32)
        W_1 = tf.Variable(tf.float32)
        b_1 = tf.Variable(tf.float32)
        W_2 = tf.Variable(tf.float32)
        b_2 = tf.Variable(tf.float32)
        
        # model architecture
        
        # question layer
        P_layer= GRU_layer(num_units=100,num_layers=120,input_data=Passage)
        P_OUT,P_STATE = tf.nn.dynamic_rnn(P_layer, data, dtype=tf.float32)
        # answer layer
        Q_layer= GRU_layer(num_units=100,num_layers=120,input_data=Question)
        Q_OUT,Q_STATE = tf.nn.dynamic_rnn(Q_layer, data, dtype=tf.float32)
        # initial memory state
        memory = Question
        # episodic memory
                      
        return 0
    
    def GRU_layer(self,num_units,num_layers,input_data):
        cells = []
        for _ in range(num_layers):
            cell = tf.contrib.rnn.GRUCell(num_units)  # Or LSTMCell(num_units)
            cells.append(cell)
        cell = tf.contrib.rnn.MultiRNNCell(cells)
        
        return cell
        
    def scalar_gate_value(self,memory,question,concept,W_b,W_1,b_1,W_2,b_2):
        
        z_vector = tf.concat(concept,memory)
        z_vector = tf.concat(z_vector,question)
        z_vector = tf.concat(z_vector,tf.multiply(concept,question))
        z_vector = tf.concat(z_vector,tf.multiply(concept,memory))
        z_vector = tf.concat(z_vector,tf.subtract(concept-question))
        z_vector = tf.concat(z_vector,tf.subtract(concept-memory))
        z_vector = tf.concat(z_vector,tf.tensordot(concept,tf.matmul(W_b,question)))
        z_vector = tf.concat(z_vector,tf.tensordot(concept,tf.matmul(W_b,memory)))
        g_1 =  tf.nn.relu(tf.add(tf.matmul(W_1,z_vector),b_1))
        g_scalar =  tf.nn.relu(tf.add(tf.matmul(W_2,g1),b_2))
        
        return g_scalar
    
    def episodic_memory(self,concepts,memory,question):
        scalar_gated_values = []
        for i in concepts:
            scalar_gated_values.append(scalar_gate_value(memory,question,i))
            
        
            
        
        
        
        
        

In [8]:
# TextBlob(str(TextBlob(df_contxt['context'][1]).sentences[0])).words

In [9]:
# df_contxt

In [10]:
# df_qas

In [45]:
def QA2V(context_no,question_no):
    passage = TextBlob(df_contxt['context'][context_no])
    question = TextBlob(df_qas['Question'][question_no])
    if df_qas['is_impossible'][question_no]==False:
        answer = TextBlob(df_qas['Answer_text'][question_no])
        beginning = passage[:df_qas['Answer_start'][question_no]]
        end = passage[df_qas['Answer_start'][question_no]+len(answer):]  # verified
        
        
#         print(answer.words,'\n',len(answer.words),'\n',len(beginning.words),'\n',len(end.words))
    print(len(passage.words),'\n',len(beginning.words),'\n',len(answer.words),'\n',len(end.words))
        
    

In [46]:
QA2V(1,1)

113 
 39 
 4 
 70


In [2]:
# TextBlob(str(TextBlob(df_contxt['context'][1]).sentences[0])+" E-O-S").words

In [6]:
x = np.array([1,2,3,4])
y = np.array([5,6,7,8])
x*y

array([ 5, 12, 21, 32])